In [1]:
// Please run this cell before the lecture

#include <iostream>
#include <stdexcept>

# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies, Cálin Georgescu, Jingya Li (Numerical Analysis, DIAM)

Lecture 3

# Overview

During last several lectures, we learned __polymorphism__, that is, inheritance of one class from another class

- Implementation of common functionality in **base class** (possibly realised as **abstract class** that cannot even be instantiated)
- Derivation of specialised class(es) from the base class that
    - implement the missing functionality (**pure virtual functions**)
    - override generic functionality by specialised variants (**virtual functions**)
    - reuse all other functionality from the base class

# Overview

Today, a more careful view on polymorphism
- **Static polymorphism**: static binding/method overloading
- **Dynamic polymorphism**: dynamic binding/method overriding
Auto functionality of C++11/14/17.

**Template meta programming**
- A powerful variant of static polymorphism

# Quiz: Guess what happens 

In [2]:
struct Base {
  virtual void hello() const = 0;
  virtual ~Base() {} // virtual destructor for proper cleanup
};

struct Derived : public Base {
  void hello() const override // Correct function name to override the base class method
  {
    std::cout << "Hallo\n";
  }
};

# Quiz: Guess what happens

Run the main function to verify your guess

In [3]:
Derived d; // Create an instance of Derived
Base* ptr = &d; // Pointer to Base type, pointing to Derived instance

ptr->hello(); // Virtual dispatch ensures Derived::hello() is called

Hallo


# Discussion

Is there another way to call the function? Think about what we have learned in the past two weeks.

**HINT:** the above example called the function by using a pointer, what we have learned at the same time with pointer?

# Discussion

Is there another way to call the function? Think about what we have learned in the past two weeks.

In [4]:
Derived d;     // Create an instance of Derived
Base& ref = d; // Reference to Base type, referring to Derived instance

ref.hello();

Hallo


# Quiz: Guess what happens

```c++
struct Base {
  virtual void hello() const = 0;
};

struct Derived : public Base {
  void hello() const
  {
    std::cout << “Hallo\n”;
  }
};
```

# Quiz: Guess what happens

```c++
struct Base {
  virtual void hello() const = 0;
};

struct Derived : public Base {
  void hello() const
  {
    std::cout << “Hallo\n”;
  }
};
```
- Structurally correct in terms of C++ syntax.
- `Derived` properly overrides the pure virtual function `hello()` from `Base`, allowing instances of `Derived` to be created and used polymorphically through pointers or references to `Base`.

# Quiz: Guess what happens

```c++
struct Base {
  virtual void hello() const {
    std::cout << "Hello\n";
  }
};

struct Derived : public Base {
  void hallo() const //Good luck with debugging
  {
    std::cout << "Hallo\n";
  }
};
```

# Quiz: Guess what happens

```C++
struct Base {
  virtual void hello() const {
    std::cout << "Hello\n";
  }
};
struct Derived : public Base {
  void hallo() const //Good luck with debugging
  {
    std::cout << "Hallo\n";
  }
};
```
- The Derived struct inherits from `Base` and introduces a new function `hallo()` that prints `Hallo` to the standard output.
- However, `Derived` does not override the `hello()` function from Base; it simply provides an additional function.

# The `override` keyword

C++11 introduces the `override` keyword to explicitly state that a function from the base class shall be _overridden_

In [5]:
struct Base {
  virtual void hello() const = 0;
  virtual ~Base() {} // virtual destructor for proper cleanup
};

struct Derived : public Base {
  void hello() const override {std::cout << "Hallo\n";}
};

Derived d;    // Create an instance of Derived
d.hello();    // Calls the overridden hello() method in Derived
Base* b = &d; // Since hello() is a virtual function, you can also call it through a Base pointer
b->hello();   // This will also call Derived::hello()
              //NOTE: If the function to be overridden does not exist in the base class an error is thrown by the compiler

Hallo
Hallo


# The `override` keyword

The overriding function must <span style=color:red;>exactly match</span> the signature of the function from the base class

<code>struct Base {
virtual <span style=color:red;>void hello() const</span> { std::cout &lt&lt “Hello\n”; }
};</code>

<code>struct Derived : public Base {
<span style=color:red;>void hello() const override</span> { std::cout &lt&lt “Hallo\n”; }
};</code>

~~<code>struct Derived2 : public Base {
<span style=color:red;>void hello() override</span> { std::cout &lt&lt “Hallo\n”; }
};</code>~~

# The final keyword
C++11 introduces the <span style=color:red;>final</span> keyword to explicitly state that a
function from the base class __must not__ be overridden

<code>struct Base {
virtual void hello() const <span style=color:red;>final</span> { std::cout &lt&lt “Hello\n”; } };</code>

If a derived class tries to override the function <span style=color:red;>hello</span> an error is thrown by the compiler

# The override and final keywords
The <span style=color:red>override</span> and <span style=color:red>final</span> keywords are often used together

```cpp
struct Base {
    // Pure virtual function must be overridden in a derived class
    virtual void hello() const = 0;
  };

  struct Derived : public Base {
    // Override pure virtual function and make it final
    void hello() const override final
    { std::cout << "Hello\n"; }
  };

  struct Derived2 : public Derived {
    // Must not try to override function hello
};
```

# Overloading vs. overriding
<div id="left" style="float: left; width: 50%;">Implementation of method or operator with <span style=color:red;>identical</span> name but a <span style=color:blue;>different interface</span>, e.g.
<p><code><span style=color:blue;>double</span> <span style=color:red;>operator*</span>(<span style=color:blue;>const
Vector &amp; other</span>) <span style=color:blue;>const</span> {...}</code></p>
<h3>overloads</h3>
<p><code><span style=color:blue;>double</span> <span style=color:red;>operator*</span>(<span style=color:blue;>double
  other</span>) <span style=color:blue;>const</span> {...}</code></p>
</div>

<div id="right" style="float: right; width: 50%;">Implementation of method or operator with <span style=color:red;>identical</span> name and the <span style=color:blue;>same interface</span> in a derived class or structure, e.g.
<p><code>struct Derived : public Base {
    <span style=color:blue;>void</span> <span style=color:red;>hello</span>(...) const override {...}
};</code></p>
<h3>overrides</h3>
<p><code>struct Base {
    virtual <span style=color:blue;>void</span> <span style=color:red;>hello</span>(...) const {...};
};</code></p>
</div>   

# Overloading vs. overriding
<img src="plots/lecture3_overloadingvsoverriding.png" width="700" >

# Implementation time! Task: Calculator
Write a class (or for demonstrating purposes a hierarchy of classes) that provide(s) a member function to calculate the **sum of two and three integer values,** respectively
- Use *static* polymorphism: method overloading
- Use *dynamic* polymorphism: method overriding

# Static polymorphism
__Method overloading__ (at compile time), the <code>StaticCalc</code> has two member functions with <span style=color:red;>identical names</span> but <span style=color:blue;>different interface</span>; compiler can decide __at compile time__ which of the two functions should be used

In [6]:
struct StaticCalc {
    int sum(int a, int b) const {
        return a + b;
    }
    
    // Overloaded function to sum three integers
    int sum(int a, int b, int c) const { return sum(sum(a,b),c);}
};

StaticCalc statC;
std::cout << "Static sum of two integers: " << statC.sum(1, 2) << std::endl;
std::cout << "Static sum of three integers: " << statC.sum(1, 2, 3) << std::endl;

Static sum of two integers: 3
Static sum of three integers: 6


# Static polymorphism
__Method overloading__ (not working!)

```cpp
struct StaticCalcBug {
 int sum(int a, int b) { return a+b; }
void sum(int a, int b) { std::cout&lt&lt a+b &lt&ltstd::endl; }
};
```

- Difference must be in the __interface of the arguments passed__. Same name with different parameter lists is allowed (either the number of parameters or their types must differ).
- You cannot have two functions that only differ by their __return type__. Because the return type could be cast into another type

# Static polymorphism
__Method overloading__ (also not working!)

```cpp
struct StaticCalcBug2 {
    int sum(int a, int b) { return a+b; }
    int sum(int c, int d) { return c+d; }
  };
```

- The interface only refers to the `types` and not to the `names`

# Static polymorphism - Final counter example

__Method overloading__ (still not working!)

```cpp
struct StaticCalcBugLast {
    int sum(int a, int b) { return a+b; }
    int sum(int a, int b, int c=1) { return a+b+c; }
  };
```

- Be careful with __default values<__. Here, the compiler cannot decide __unambiguously__ which of the two functions should be used in the case `sum(arg0, arg1)`

# Rule of thumb:
__If it is not crystal clear to you which function should be used then the compiler will fail, too.__
<center><img src="plots/lecture3cat.gif" width="400" ></center>

# Static polymorphism (master version1)

__Method overloading__: decision about which method to call is made at compile time; hence the compiler can decide to __inline code__ to improve performance (no overhead due to function calls and copying of data to and from the stack!)

```C++
std::cout << statC.sum(1,2)   << std::endl;
std::cout << statC.sum(1,2,3) << std::endl;
```
 becomes

```C++
std::cout << (1+2)        << std::endl;
std::cout << ((1+2)+3)    << std::endl;
```

# Static polymorphism (master version2)

__Method overloading__: since static polymorphism takes place at compile time, the `inline` specifier can be used to explicitly ‘suggest’ to the compiler to inline the function

```cpp
struct StaticCalcMaster {
    inline int sum(int a, int b)        { return a+b; }
    inline int sum(int a, int b, int c) { return sum(sum(a,b),c); }
  };
```

# Dynamic polymorphism

__Method overriding__: reimplementation of a function inherited from a base class with new function body and same interface!

In [7]:
struct BaseCalc {
    virtual int sum2(int a, int b) { return a+b; }
    int sum3(int a, int b, int c)
        { return sum2(sum2(a,b),c); }
  };

struct DerivedCalc : public BaseCalc {
    int sum2(int a, int b) override final { return b+a; }
};

In [8]:
BaseCalc baseCalc;
DerivedCalc derivedCalc;

// Using the base class to sum two numbers
std::cout << "BaseCalc sum2: " << baseCalc.sum2(1, 2) << std::endl;

// Using the base class to sum three numbers
std::cout << "BaseCalc sum3: " << baseCalc.sum3(1, 2, 3) << std::endl;

// Using the derived class to sum two numbers, this will use the overridden method
std::cout << "DerivedCalc sum2: " << derivedCalc.sum2(1, 2) << std::endl;

// Using the derived class to sum three numbers, this will use the base
// class method sum3, which in turn will use the overridden sum2 method
// from DerivedCalc
std::cout << "DerivedCalc sum3: " << derivedCalc.sum3(1, 2, 3) << std::endl; 

BaseCalc sum2: 3
BaseCalc sum3: 6
DerivedCalc sum2: 3
DerivedCalc sum3: 6


# Dynamic polymorphism (where things can go wrong)

__Method overriding__: a common pitfall is to forget the virtual specifier in the base class to indicate that the <code>sum2</code> function from the base class can be overridden in a derived class

<code>struct BaseCalc {
    <span style=color:red;>int sum2</span>(int a, int b) { return a+b; }
    <span style=color:red;>int sum3</span>(int a, int b, int c){ return sum2(sum2(a,b),c); }
  };
  struct DerivedCalc : public BaseCalc {
    <span style=color:red;>int sum2</span>(int a, int b) { return b+a; }
};</code>

# Dynamic polymorphism (where things can go wrong)
Take home notes, read after the lecture.
Here's what happens if you forget <code>virtual</code> specifier
- __Method Hiding:__ In <code>DerivedCalc</code>, the <code>sum2</code> method hides the <code>sum2</code> method from <code>BaseCalc</code>. This means that if you have an object of <code>DerivedCalc</code> and call <code>sum2</code>, it will use <code>DerivedCalc</code>'s <code>sum2</code>. However, if you have a pointer or reference to <code>BaseCalc</code> that actually points to a <code>DerivedCalc</code> object and call <code>sum2</code>, it will still use <code>BaseCalc</code>'s <code>sum2</code>. This is because the method is not virtual and hence does not support polymorphic behavior.


- __No Polymorphism:__ Without the virtual keyword in <code>BaseCalc</code>, <code>sum2</code> in <code>DerivedCalc</code> doesn't exhibit polymorphic behavior, crucial for method overriding in C++.

<span style=color:red;><u>Rule of thumb:</u></span> These bugs are hard to find (they often remain unrecognised) and can be prevented by using the <code>override</code> keyword in C++11. It explicitly indicates that a function is intended to override a virtual function in a base class.

# Dynamic polymorphism - general suggestions
__Method overriding:__ decision about which virtual method to call is made at run time; hence inlining is not possible

Common design pattern
- Specify <span style=color:red;>expected minimal functionality</span> of a group of classes in
abstract base class via <span style=color:red;>pure virtual member functions</span>
- Implement generic common functionality of a group of classes in
abstract base class via <span style=color:red;>virtual member functions</span>
- Implement specific functionality of a particular derived class by <span style=color:red;>overriding the pure virtual member function</span>

# Example: inner product space
In linear algebra, an inner product space is a vector space $V$ that is equipped with a special mapping (inner product)
$$\langle\cdot, \cdot\rangle: V \times V \rightarrow \mathbb{R}\text{ or }\mathbb{C}$$
Inner product spaces have a naturally induced norm
$$\|x\|=\sqrt{\langle x, x\rangle}$$

# Example: inner product space
Class <code>InnerProductSpaceBase</code> __declares inner product__ as pure
virtual and __implements the naturally induced norm__

<code>struct InnerProductBase
{
    <span style=color:red;>virtual</span> double <span style=color:red;>inner_product</span>(... x,... y)<span style=color:red;> = 0</span>;
    double norm(... x) { return inner_product(x,x); }
};</code>

Derived <code>InnerProductSpace</code> class __implements inner product__

<code>struct InnerProductSpace : public InnerProductSpaceBase {
    double <span style=color:red;>inner_product</span>(... x, ... y) = { return x*y; }};</code>

# Implementation time! Task: Calculator2
Extend the calculator class so that it can handle numbers of integer, float and double type at the same time

- Prevent manual code duplication
- Prevent explicit type casting
- Make use of auto-functionality (C++11/14/17)
- Make use of template meta programming

# Vanilla implementation in C++

In [9]:
# include <iostream>
struct Calc2 {
    int sum(int a, int b) { return a + b; }
    int sum(int a, int b, int c) { return sum(sum(a, b), c); }
};

Calc2 calculator;

// Using the sum method with two arguments
std::cout << "Sum of 1 and 2: " << calculator.sum(1, 2) << std::endl;

// Using the sum method with three arguments
std::cout << "Sum of 1, 2, and 3: " << calculator.sum(1, 2, 3) << std::endl;

Sum of 1 and 2: 3
Sum of 1, 2, and 3: 6


# Automatic return type deduction (C++11)
- Explicit definition of the function return type
  ```cpp
  int sum(int a, int b)
      { return a+b; }
  ```

- Automatic function return type (since C++11)
  ```cpp
  auto sum(int a, int b) -> decltype(a+b)
       { return a+b; }
  ```
- By using `decltype`, the __return type__ of the sum function is determined automatically as the type of `operator+(a,b)`

# Automatic return type deduction

The `decltype` specifier (C++11) queries the type of an expression

In [10]:
#include <iostream>
struct Calc2 {
    
    auto sum(int a, int b) -> decltype(a + b) {
        return a + b;
    }
    
    auto sum(int a, int b, int c) -> decltype(sum(sum(a, b), c)) {
        return sum(sum(a, b), c);
    }
};

Calc2 C;
std::cout << C.sum(1, 2) << std::endl;   // Outputs: 3
std::cout << C.sum(1, 2, 3) << std::endl; // Outputs: 6

3
6


@0x7ffb425f6de0

# Automatic type deduction (C++14)

C++14 deduces the type of parameters automatically

```cpp
auto sum(int a, int b) // no -> decltype(...) needed
    { return a+b; }

auto sum(int a, int b, int c) // no -> decltype(...) needed
    { return sum(sum(a,b),c); }
```
   
__Remark:__ This C++14 feature helps to improve readability of the code and prevents deduction errors (due to forgotten/ inconsistent deduction rule by the programmer) but it does not solve the problem of being able to pass arguments of different types to the same function

# Function templates

Template meta programming is the standard technique to deal with arbitrary (= generic) function parameters

__Function templates:__ allow you to implement so-called parameterized functions for generic parameter types

<code><span style=color:red;>template&lt</span><span style=color:purple;>typename R</span>, <span style=color:blue;>typename A, typename B</span>>
<span style=color:purple;>R</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b)
{return a+b; }</code>

# Function templates

Types must be specified explicitly when the function is called

<code><span style=color:purple;>int</span> s1 = <span style=color:red;>sum&lt</span> <span style=color:purple;>int</span>, <span style=color:blue;>int, int</span>>(1, 2);
<span style=color:purple;>double</span> s2 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>, <span style=color:blue;>double, int</span>>(1.2, 2);
<span style=color:purple;>double</span> s3 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>,<span style=color:blue;> float, double</span>>(1.4, 2.2);
</code>
This can be slightly simplified by using the __auto specifier__

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span> <span style=color:purple;>int</span>, <span style=color:blue;>int, int</span>>(1, 2);
<span style=color:purple;>auto</span> s2 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>, <span style=color:blue;>double, int</span>>(1.2, 2);
<span style=color:purple;>auto</span> s3 = <span style=color:red;>sum&lt</span><span style=color:purple;>double</span>,<span style=color:blue;> float, double</span>>(1.4, 2.2);
</code>

# Function templates

C++11: __automatic return type deduction__

<code><span style=color:red;>template&lt</span> <span style=color:blue;>typename A, typename B</span>>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b)<span style=color:purple;> –> decltype(a+b)</span>{ return a+b; }</code>

C++14: __automatic type deduction__

<code><span style=color:red;>template&lt</span> <span style=color:blue;>typename A, typename B</span>>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b){ return a+b; }</code>

Usage

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span><span style=color:blue;>int, int</span>>(1, 2);</code>

# Function templates

How to convert this function into a templated function

```C++
int sum(int a, int b, int c)
{
      return sum(sum(a,b), c);
}
```  

# Function templates

Use explicit return type parameter (ugly!) -> NO

<code><span style=color:red;>template&lt</span><span style=color:purple;>typename R,</span><span style=color:blue;> typename A, typename B, typename C</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
  {
      return <span style=color:red;>sum&lt</span><span style=color:purple;>R</span><span style=color:blue;>,C</span><span style=color:red;>></span>(<span style=color:red;>sum&lt</span><span style=color:blue;>A,B</span><span style=color:red;>></span>(a,b), c);
  }</code>
 
Guess what this function call will return

<code>auto s1 = sum<span style=color:red;>&lt</span> <span style=color:purple;>int,</span> <span style=color:blue;>double,double,double</span><span style=color:red;>></span>(1.1,2.2,3.3)</code>

# Function templates

Use a smart combination of templates and auto specifier

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b) <span style=color:purple;>-> decltype(a+b) // omit in C++14</span>
{return a+b; }
</code>

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B, typename C</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
{
    return <span style=color:red;>sum&lt</span><span style=color:purple;>decltype(sum&ltA,B>(a,b))</span>,C<span style=color:red;>></span> (<span style=color:red;>sum&lt</span><span style=color:blue;>A,B</span><span style=color:red;>></span>(a,b), c);
}</code>

# Function templates
But wait, C++ <u>can</u> deduce the type of the template
argument from the given argument automatically

<code><span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b) <span style=color:purple;>-> decltype(a+b) // omit in C++14</span>
{return a+b; }
</code>

<code>
<span style=color:red;>template&lt</span><span style=color:blue;>typename A, typename B, typename C</span><span style=color:red;>></span>
<span style=color:purple;>auto</span> sum(<span style=color:blue;>A</span> a, <span style=color:blue;>B</span> b, <span style=color:blue;>C</span> c)
{
    return <span style=color:red;>sum</span>(<span style=color:red;>sum</span>(a,b), c);
}</code>

# Function templates

We can call the generic sum functions as follows:

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum&lt</span><span style=color:blue;>int, int</span><span style=color:red;>></span> (1, 2);
<span style=color:purple;>auto</span> s2 = <span style=color:red;>sum&lt</span><span style=color:blue;>double, int</span><span style=color:red;>></span>  (1.2, 2);
<span style=color:purple;>auto</span> s3 = <span style=color:red;>sum&lt</span><span style=color:blue;>float, double</span><span style=color:red;>></span>(1.4f, 2.2);</code>

Or, even simpler, as follows:

<code><span style=color:purple;>auto</span> s1 = <span style=color:red;>sum</span> (1, 2);
<span style=color:purple;>auto</span> s2 = <span style=color:red;>sum</span> (1.2, 2);
<span style=color:purple;>auto</span> s3 = <span style=color:red;>sum</span> (1.4f, 2.2)</code>

# Function templates

- Since the compiler needs to duplicate code and substitute A,B,C for each combination of templated types both the compile time and the size of the executable will increase
- Template meta programming is simplest if the code resides in header files only; later we will see how to use template meta-programming together with pre-compiled libraries

# Implementation time! Task: generic Vector class

Write a Vector class that can store real values and complex values and supports the following operations:
- Addition of two vectors of the same length
- Multiplication of a vector with a scalar
- Dot product of a vector with another one
In all operations, the involved objects (vectors/scalar) can be of different types, e.g., double/float

# Vector class prototype

Implementation of Vector-of-double class

In [71]:
#include <iostream>
class Vector{
private:
    double* data;
    int n;
public:
//----------------------------------- Constructors below --------------------------------------------------//
    Vector() : n(0), data(nullptr) {} // Default constructor

    Vector(int n) : n(n), data(new double[n]) {} // Constructor with size

    ~Vector(){ n=0;  delete[] data; }// Destructor
//----------------------------------- method and operators --------------------------------------------------//

    // Addition (+=) operator
    Vector& operator+=(const Vector& other) {
        if (n != other.n) {
            throw std::invalid_argument("Vectors must be of the same size");
        }
        for (auto i = 0; i < n; i++) {
            data[i] += other.data[i];
        }
        return *this;
    }

    // Scalar multiplication (*=) operator
    Vector& operator*=(double scalar) {
        for (int i = 0; i < n; i++) {
            data[i] *= scalar;
        }
        return *this;
    }

    // Dot product
    double dot(const Vector& other) const {
        if (n != other.n) {
            throw std::invalid_argument("Vectors must be of the same size");
        }
        double sum = 0;
        for (auto i = 0; i < n; i++) {
            sum += data[i] * other.data[i];
        }
        return sum;
    }
    // Subscript operator
    double& operator[](int index) {
        if (index >= n || index < 0) {
            throw std::out_of_range("Index out of range");
        }
        return data[index];
     }
};

# Brainstorming
__Function templates__ alone will not help since the type of a class attribute needs to be templated -&gt; <span style=color:red;>class templates</span>

Some member functions can be implemented generically, e.g., addition of two vectors and multiplication of a vector with a scalar value since they are the same for all types

Some member functions must be implemented in different manners for real and complex values -&gt; <span style=color:red;>specialisation</span>

$$x \cdot y=\sum_{i=1}^n x_i y_i, \quad x, y \in \mathbb{R}, \quad x \cdot y=\sum_{i=1}^n x_i \bar{y}_i \quad x, y \in \mathbb{C}$$

# Class template: anything vector class
Implementation of Vector-of-<span style=color:blue;>anything</span> class --  creating a generic vector that can store elements of any type.

<u>CAUTION:</u> We are going to leave the presentation mode to have an overview of the entire program (without the main function).


In [72]:
#include <iostream>
#include <stdexcept>

template<typename T>
class Vector {
private:
    T* data;
    int n;

public:
//--------------------------------------Constructors below-----------------------------------------//
    // Default constructor
    Vector() : data(nullptr), n(0) {}

    // Constructor with size
    Vector(int n) : n(n), data(new T[n]()) {}

    // Destructor
    ~Vector() {
        delete[] data;
    }

//----------------------End of constructors, go to next page for explaination----------------------//
    

//------------------------------------Methods and operators below-----------------------------------------// 
    // Addition (+=) operator
    Vector<T>& operator+=(const Vector<T>& other) {
        if (n != other.n) {
            throw std::invalid_argument("Vectors must be of the same size");
        }
        for (int i = 0; i < n; i++) {
            data[i] += other.data[i];
        }
        return *this;
    }
    
    // Scalar multiplication (*=) operator
    Vector<T>& operator*=(T scalar) {
        for (int i = 0; i < n; i++) {
            data[i] *= scalar;
        }
        return *this;
    }
    
    // Dot product
    T dot(const Vector<T>& other) const {
        if (n != other.n) {
            throw std::invalid_argument("Vectors must be of the same size");
        }
        T sum = 0;
        for (int i = 0; i < n; i++) {
            sum += data[i] * other.data[i];
        }
        return sum;
    }
//------------------Combined class template and function template below------------------------// 
    template<typename S>
    Vector<T>& operator*=(S scalar){
        for (auto i=0; i<n; i++)
            data[i] *= scalar;
        return *this;
    }
    
// Non-const subscript operator
    T& operator[](int index) {
        if (index < 0 || index >= n) {
            throw std::out_of_range("Index out of range");
        }
        return data[index];
    }

// Const version of subscript operator
    const T& operator[](int index) const {
        if (index < 0 || index >= n) {
            throw std::out_of_range("Index out of range");
        }
        return data[index];
    }
};

# Class template
Template parameter must be explicitly specified
```C++
Vector<int> x(10);  // Vector-of-int with length 10
Vector<double> y;   // Empty Vector-of-double
Vector<float> z(5); // Vector-of-float with length 5    
```

Remark: if you want to pass a Vector-of-<span style=color:blue;>anything</span> to a function in the templated class Vector you have to write

```C++
Vector<T>  v
Vector<T>& v
```

instead of

```C++
Vector  v
Vector& v
```

# Class template
With the **class template parameter T** we can do

In [70]:
Vector<int> x1(5);
Vector<int> x2(5);

// Initialize x1 and x2 with some values
for (int i = 0; i < 5; ++i) {
    x1[i] = i + 1;  // x1 = [1, 2, 3, 4, 5]
    x2[i] = 2 * (i + 1);  // x2 = [2, 4, 6, 8, 10]
}

x1 += x2; // x1 = [3, 6, 9, 12, 15]
x1 *= 2;  // x1 = [6, 12, 18, 24, 30]
int dot_product = x1.dot(x2);  // Dot product of x1 and x2

std::cout << "Dot product:" << dot_product;

Dot product:660

How about?

```C++
Vector<int> x2(5);
x2 *= (double)1.2;
```

# Intermezzo

<code><span style=color:red;>template</span><span style=color:red;>&lt</span><span style=color:blue;>typename T</span><span style=color:red;>></span>
class Vector {
...
      <span style=color:red;>template&lt</span><span style=color:purple;>typename S</span><span style=color:red;>></span>
      Vector&lt<span style=color:blue;>T</span>>&amp operator*=(<span style=color:purple;>S</span> scalar)
      {
          for (auto i=0; i&ltn; i++)
              data[i] *= scalar;
          return *this;
      }
};
</code>

# Intermezzo
At first glance, this seems to be more flexible

<code>Vector&lt<span style=color:blue;>double</span>> x1(5); x1 = x1*(<span style=color:purple;>int</span>)(5);</code>

But be really careful since strange things can happen

<code>Vector&lt<span style=color:blue;>int</span>> x1(5); x1 = x1*(<span style=color:purple;><span style=color:purple;>double</span></span>)(5);</code>

<u>Rule of thumb:</u> before using extensive template meta programming like this think about all(!) implications

# Specialization
The dot product needs special treatment since

<code><span style=color:blue>T</span> dot(const Vector&lt<span style=color:blue>T</span>>&amp other) const
    {
        <span style=color:blue>T</span> d=0;
        for (auto i=0; i&ltn; i++)
             d += data[i]*other.data[i];
        return d; }</code>
- Lacks the complex conjugate of other and yields the wrong return type in case of complex-valued vectors

<u>Remedy:</u> implement a specialised variant for this case

# Specialization
Generic dot product implemented in Vector class

<code>#include &ltcomplex>
<span style=color:red>template</span>&lt<span style=color:blue>typename T</span>>
class Vector { ...
   <span style=color:blue>T</span> dot(const Vector&lt<span style=color:blue>T</span>>&amp other) const {...}
};</code>

This function is used whenever no specialized implementation for a concrete type is available

# Specialization
 Specialized dot product for Vectors-of-<span style=color:blue;>complex</span>-<span style=color:purple;>complex</span>
 
<code><span style=color:red;>template&lt></span>
<span style=color:blue;>std::complex&lt</span><span style=color:purple;>float</span><span style=color:blue;>></span> Vector&lt <span style=color:blue;><span style=color:blue;>std::complex&lt</span></span><span style=color:purple;>float</span><span style=color:blue;>></span> >::
    dot(const Vector&lt <span style=color:blue;>std::complex&lt</span><span style=color:purple;>float</span>>>&amp other) const {
          <span style=color:blue;>std::complex&lt</span><span style=color:purple;>float</span><span style=color:blue;>></span> d=0;
          <span style=color:red;>// special treatment of dot product</span>
          for (auto i=0; i&ltn; i++)
              d += data[i]*<span style=color:red;>std::conj(</span>other.data[i]<span style=color:red;>)</span>;
          return d;
}</code>

# Specialization

Current implementation yields

<code>Vector&lt<span style=color:blue;>float</span>> x1(5), x2(5);
auto x1.dot(x2); // calls generic implementation</code>

<code>Vector&lt<span style=color:blue;>std::complex&lt</span><span style=color:purple;>float</span><span style=color:blue;>></span> x1(5), x2(5);
auto x1.dot(x2); // calls generic implementation</code>

<code>Vector&lt<span style=color:blue;>std::complex&lt</span><span style=color:purple;>double</span><span style=color:blue;>></span> > z1(5), z2(5);
auto z1.dot(z2); // calls generic implementation</code>

<code>auto x1.dot(y1); // does not even compile</code>

# Outlook on next session

C++ allows you to **partially specialize** class templates

<code><span style=color:red;>template&lt</span><span style=color:purple;>typename S</span><span style=color:red;>></span>
<span style=color:blue;>std::complex&lt</span><span style=color:purple;>S</span>> Vector&lt<span style=color:blue;>std::complex&lt</span><span style=color:purple;>S</span>> >::
    dot(const Vector&lt<span style=color:blue;>std::complex&lt</span><span style=color:purple;>S</span>> > other) const { 
        <span style=color:blue;>std::complex&lt</span><span style=color:purple;>S</span>> d=0;
        for (auto i=0; i&ltn; i++)
              d += data[i]*<span style=color:red;>std::conj(</span>other.data[i]<span style=color:red;>)</span>;
        return d;
}</code>

Note that this code will not compile. We will see why and learn remedies. Welcome to where template magic begins!